Who to blame for these plots: Alex Payne

This notebook is used to generate the plots for the paper, and is based on csv file `scores_by_molecule_and_method.csv` generated in the previous notebook `poses_rmsd_distribution.ipynb`.

In [ ]:
import pandas as pd
import base64
from rdkit import Chem 
from rdkit.Chem.Scaffolds import MurckoScaffold
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
entry_categories = {
    "competition-prediction-8oFF30AHg20eUKrnRQzUE": "CF",
    "competition-prediction-Edd645cpFboFZRsUm20em": "DL",
    "competition-prediction-tXLnHU72dFljtgWAuvh8Z": "PB",
    "competition-prediction-late-im0HtqthslCqgR1BHgf9b": "CF",
    "competition-prediction-sqr7BVpSxy4NSgljcEgl8": "PB",
    "competition-prediction-t4heZQ05hXlJ6vcfhZjP2": "PB",
    "competition-prediction-GL85gNve6u38L33Q2wUiJ": "DL",
    "competition-prediction-Kec3G0H9KJGWfRouaHDLQ": "PB",
    "competition-prediction-JPiM6etVV4SNygnvAZqj7": "CF",
    "competition-prediction-M0ePYflVVxiF79RnRcCkC": "PB",
    "competition-prediction-S9yhyrYVraSy6vZ3u0iwS": "DL",
    "competition-prediction-pxXgSbG31dfMbqohnMnfP": "CF",
    "competition-prediction-DLGeSzUxB29MqK9pnBawM": "PB",
    "competition-prediction-8ygjELYPeCo3or8AS6Xom": "DL",
    "competition-prediction-o8n3Fgdevuj8th3FfUNu1": "DL",
    "competition-prediction-TPDYWfhEw4nYp81Kd5s1W": "PB",
    "competition-prediction-th7lSmtpv1tMxmNjYqCJU": "PB",
    "competition-prediction-kMD7A97Q9b7wnRH8Ze9e": "PB",
    "competition-prediction-TSWQyQgwNyUbdp5EaGbmU": "PB",
    "competition-prediction-SXMdEmccFmbNBtiVCZvp9": "CF",
    "competition-prediction-JLy17Fu87yM2ch741quDp": "PB",
    "competition-prediction-MoEWcKkwYkPeFHrw3qaWd": "PB",
    "competition-prediction-sKvyY8FZzDDX1jaFpBG8t": "CF",
    "competition-prediction-GZRD1499Q0k3ejSX7wR22": "PB",
    'competition-prediction-kMD7A97Q9b7wnRH8Ze9e9': "PB",
}

In [ ]:
BASELINE_ID = "competition-prediction-DLGeSzUxB29MqK9pnBawM"

In [ ]:
# Define method category labels
method_labels = {"CF": "Co-Folding", "DL": "Deep-Learning", "PB": "Physics-based"}

In [ ]:
def deserialize_rdkit_mol(b64_string: str) -> Chem.Mol:
    mol_bytes = base64.b64decode(b64_string)
    return Chem.Mol(mol_bytes)

In [ ]:
def serialized_list_to_rdkit(data: list[str]) -> list[Chem.Mol]:
    mols = []
    for d in data:
        mols.append(deserialize_rdkit_mol(d))
    return mols

In [ ]:
def get_generic(mol):
    scaff = MurckoScaffold.GetScaffoldForMol(mol)
    scaff = MurckoScaffold.MakeScaffoldGeneric(scaff)
    return Chem.MolToSmiles(scaff)

In [ ]:
scores_by_scaffold_and_method = pd.read_csv("scores_by_molecule_and_method.csv")

In [ ]:
single_method = scores_by_scaffold_and_method[scores_by_scaffold_and_method["Method"] == scores_by_scaffold_and_method["Method"].unique()[0]]

In [ ]:
scaff_counts = single_method.groupby(["Scaffold"]).count()

In [ ]:
sc = scaff_counts.sort_values("RMSD", ascending=False)["RMSD"].reset_index()

In [ ]:
sc.columns=["Scaffold", "Count"]

In [ ]:
small = sc[sc["Count"] > 5]

In [ ]:
scores_by_scaffold_and_method_simple_scaffolds = scores_by_scaffold_and_method[scores_by_scaffold_and_method["Scaffold"].isin(small["Scaffold"].tolist())]

In [ ]:
df = scores_by_scaffold_and_method_simple_scaffolds.copy()

In [ ]:
df["Success"] = df["RMSD"] < 2

In [ ]:
n_correct = df[["Method", "Scaffold", "Protein", "Success"]].groupby(["Method", "Scaffold", "Protein"]).sum()["Success"]

In [ ]:
total = df[["Method", "Scaffold", "Protein", "Success"]].groupby(["Method", "Scaffold", "Protein"]).count()["Success"]

In [ ]:
success_rate_by_scaffold = n_correct / total

In [ ]:
success_rate_by_scaffold = success_rate_by_scaffold.reset_index()

In [ ]:
scores_by_scaffold_and_method["Category"] = scores_by_scaffold_and_method.Method.apply(lambda x: entry_categories.get(x, None))
scores_by_scaffold_and_method["Method Type"] = scores_by_scaffold_and_method.Category.apply(lambda x: method_labels.get(x, None))

In [ ]:
scores_by_scaffold_and_method["RDKitMol"] = scores_by_scaffold_and_method.SMILES.apply(lambda x: Chem.MolFromSmiles(x))

# add series information

In [ ]:
lead = Chem.MolFromSmiles("O=C1CCCN1c(cnc2)c3c2cccc3")
backup = Chem.MolFromSmiles("NC(CC1=CN=CC2=CC=CC=C12)=O")
## Write out scaffolds
scaffs = {"lead": lead, "backup": backup}

## Get which series each molecule is in
scores_by_scaffold_and_method["IS_LEAD"] = scores_by_scaffold_and_method.RDKitMol.apply(
    lambda x: x.HasSubstructMatch(scaffs["lead"]))
scores_by_scaffold_and_method["IS_BACKUP"] = scores_by_scaffold_and_method.RDKitMol.apply(
    lambda x: x.HasSubstructMatch(scaffs["backup"]))


def get_series(mol):
    if mol.HasSubstructMatch(scaffs["lead"]):
        return "Lead"
    if mol.HasSubstructMatch(scaffs["backup"]):
        return "Backup"
    else:
        return "Misc"


scores_by_scaffold_and_method["Series"] = scores_by_scaffold_and_method.RDKitMol.apply(get_series)

## calculate success rate vs series

In [ ]:
df = scores_by_scaffold_and_method.copy()
df["Success"] = df["RMSD"] < 2
n_correct = df[["Method", "Series", "Protein", "Success"]].groupby(["Method", "Series", "Protein"]).sum()["Success"]
total = df[["Method", "Series", "Protein", "Success"]].groupby(["Method", "Series", "Protein"]).count()["Success"]
success_rate = n_correct / total
success_rate = success_rate.reset_index()

In [ ]:
success_rate["method_type"] = success_rate["Method"].apply(lambda x: entry_categories.get(x, None))

In [ ]:
success_rate["Method Type"] = success_rate["method_type"].apply(lambda x: method_labels.get(x, None))

## calculate success rate vs method

In [ ]:
df = scores_by_scaffold_and_method.copy()
df["Success"] = df["RMSD"] < 2
n_correct = df[["Method Type", "Method", "Protein", "Success"]].groupby(["Method Type", "Method", "Protein"]).sum()["Success"]
total = df[["Method Type", "Method", "Protein", "Success"]].groupby(["Method Type", "Method", "Protein"]).count()["Success"]
success_rate_by_method = n_correct / total
success_rate_by_method = success_rate_by_method.reset_index()

# plotting defaults

In [ ]:
plt.style.use('default')
sns.set_theme(style="ticks")

# Define colors
lead_color = "#1E88E5"
backup_color = "#FFC107"
misc_color = "#004D40"

# Reorder the proteins
protein_order = ["SARS-CoV-2 Mpro", "MERS-CoV Mpro"]
series_order = ["Lead", "Backup", "Misc"]

# Define method category labels
method_labels = {"CF": "Co-Folding", "DL": "Deep-Learning", "PB": "Physics-based"}

# Create custom palette
palette = {"Lead": lead_color, 
           "Backup": backup_color, 
           "Misc": misc_color,
           "Co-Folding": "#b51963",
           "Deep-Learning": "#0073e6",
           "Physics-based": "#5ba300"}

lead_success_title = "Success Rate on Lead Series"
backup_success_title = "Success Rate on Backup Series"

### KDE

In [ ]:
# plotdf = success_rate[~(success_rate["Series"] == "Misc")]
plotdf = success_rate
g = sns.displot(plotdf, x="Success", hue="Series", palette=palette, col="Protein", col_order=protein_order, kind="kde", fill=True, alpha=0.5, hue_order=series_order)

# Save the figure
g.fig.savefig('success_rate_distribution_kde.png',
              bbox_inches='tight',
              dpi=300)

# success rate by series 

In [ ]:
sns.set_theme(style="white")
# plotdf = success_rate[~(success_rate["Series"] == "Misc")]
plotdf = success_rate

palette = {"Lead": lead_color, 
           "Backup": backup_color, 
           "Misc": misc_color,}

g = sns.displot(plotdf, 
                x="Success", 
                hue="Series", 
                palette=palette, 
                col="Protein", 
                col_order=protein_order, 
                kind="ecdf", 
                hue_order=series_order,
                height=4,
                aspect=1)

# Update subplot titles
g.set_titles(col_template="{col_name}")

# Update line thickness
for ax in g.axes[0]:
    for line in ax.lines:
        line.set_linewidth(2)

sns.move_legend(g, "upper left", bbox_to_anchor=(0.1, 0.9))

# Remove x-axis ticks and labels
for ax in g.axes[0]:
    ax.set_xticks([])
    ax.set_xticklabels([])
    ax.set_xlabel('')

for ax in g.axes[0]:
    ax.set_xlim(0, 1.05)
    ax.set_ylim(0, 1.05)
#     ax.set_aspect('equal')

# Save the figure
g.fig.savefig('success_rate_distribution_ecdf.pdf',
              bbox_inches='tight',
              dpi=300)

## pairplot from backup to lead for each type of entry

In [ ]:
success_rate_lead = success_rate[success_rate["Series"] == "Lead"]
success_rate_backup = success_rate[success_rate["Series"] == "Backup"]

In [ ]:
lead_vs_backup = pd.merge(success_rate_lead, success_rate_backup, on=["Method", "Protein", "Method Type"], suffixes=["_Lead", "_Backup"]).drop(columns=["Series_Lead", "Series_Backup"])

In [ ]:
sns.scatterplot(lead_vs_backup, x="Success_Lead", y="Success_Backup", hue="Method Type", style="Protein", style_order=protein_order)

In [ ]:
lead_vs_backup[lead_success_title] = lead_vs_backup["Success_Lead"]
lead_vs_backup[backup_success_title] = lead_vs_backup["Success_Backup"]
palette = {"Co-Folding": "#51ac15",
           "Deep-Learning": "#2e70f0",
           "Physics-based": "#c03868"}
g = sns.relplot(data=lead_vs_backup, x=lead_success_title, y=backup_success_title, 
                hue="Method Type", col="Protein", col_order=protein_order, palette=palette
                )

# Update subplot titles and move legend
g.set_titles(col_template="{col_name}")
g._legend.remove()  # Remove the original legend
g.fig.legend(loc='upper left', bbox_to_anchor=(0.125, 0.9))

for ax in g.axes[0]:
    ax.set_xlim(-0.1, 1)
    ax.set_ylim(-0.1, 1)
    # ax.set_aspect('equal')
# save figure
# Save the figure
g.fig.savefig('success_rate_lead_vs_backup_comparison.png',
              bbox_inches='tight',
              dpi=300)

# plot by method type

In [ ]:
plotdf = success_rate_by_method.copy()
sns.set_style("white")

palette = {"Co-Folding": "#51ac15",
           "Deep-Learning": "#2e70f0",
           "Physics-based": "#c03868"}

g = sns.displot(plotdf, 
                x="Success", 
                hue="Method Type", 
                col="Protein", 
                col_order=protein_order, 
                kind="ecdf", 
                palette=palette,
                height=4,
                aspect=1)
# Update subplot titles
# g.set_titles(col_template="{col_name}")

# Remove subplot titles
g.set_titles(col_template="")

# Update line thickness
for ax in g.axes[0]:
    for line in ax.lines:
        line.set_linewidth(2)

sns.move_legend(g, "upper left", bbox_to_anchor=(0.075, 1))

for ax in g.axes[0]:
    ax.set_xlim(0, 1.05)
    ax.set_ylim(0, 1.05)
    # ax.set_aspect('equal')

g.fig.savefig('success_rate_vs_method_type_ecdf.pdf',
              bbox_inches='tight',
              dpi=300)

# plot by scaffold

In [ ]:
# plotdf = success_rate[~(success_rate["Series"] == "Misc")]
plotdf = success_rate_by_scaffold
g = sns.displot(plotdf, x="Success", hue="Scaffold", col="Protein", col_order=protein_order, kind="kde", fill=False)

# Save the figure
g.fig.savefig('success_rate_distribution_by_scaffold_kde.png',
              bbox_inches='tight',
              dpi=300)

In [ ]:
plotdf = success_rate_by_scaffold
g = sns.displot(plotdf, x="Success", hue="Scaffold", col="Protein", col_order=protein_order, kind="ecdf")

# Save the figure
g.fig.savefig('success_rate_distribution_by_scaffold_ecdf.png',
              bbox_inches='tight',
              dpi=300)

# Check if it's true that only a few of the methods tested on MISC

In [ ]:
success_rate.groupby(["Protein", "Series"]).nunique()

In [ ]:
scores_by_scaffold_and_method.groupby(["Protein", "Series"]).nunique()

### it's just that there's only 4 ligands in the "misc" category for MERS, so success rate can only be a few discrete numbers